In [1]:
# --- Local Setup ---
import os
import sys
from pathlib import Path

# Add repository root to path
# Assuming notebook is in notebooks/ and repo root is one level up
repo_root = Path("..").resolve()
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

# Change working directory to repo root so config files are found correctly
os.chdir(repo_root)

In [8]:
# setup Environment & Config
import yaml
from src.config import load_settings

# Define local data paths
# User specified ./data/ contains data, models, results, runs, yoloConfig
# We assume this folder is in the repository root
catnip_data_root = repo_root / "catnip-data"
data_dir = catnip_data_root / "data"

# Override data path to point to local synced folder
os.environ["CATNIP_PATHS_DATA"] = str(data_dir)

settings = load_settings()
print(f"Data Root: {settings.paths.data}")
print(f"Manga Dir: {settings.paths.manga_dir}")
print(f"Annotations Dir: {settings.paths.annotations_dir}")



Data Root: data
Manga Dir: catnip-data\data\manga
Annotations Dir: data\annotations


In [ ]:
# prepare Dataset for YOLO
from src.training.preparation import safe_symlink

data_root = settings.paths.data
images_link = data_root / "images"
labels_link = data_root / "labels"

if data_root.exists():
    safe_symlink(settings.paths.manga_dir, images_link)
    safe_symlink(settings.paths.annotations_dir, labels_link)
else:
    print(f"Warning: Data root {data_root} does not exist. Please ensure catnip-data is synced.")

In [ ]:
# convert label studio json to yolo format
from src.convert_labels import convert_label_studio_to_yolo

# Update path to local export
# Assuming exports are in the repo's data folder
json_file = repo_root / "data" / "ls-exports" / "251226v1.json"

if json_file.exists():
    convert_label_studio_to_yolo(json_file, settings.paths.annotations_dir, { "izutsumi": 0, "izutsumi_face": 1 })
else:
    print(f"Label Studio export not found at {json_file}")

In [ ]:
# generate or load training list based on available labels
from src.training.preparation import generate_training_list

# define config directory mapping to ./catnip-data/yoloConfig/
config_dir = catnip_data_root / "yoloConfig"
config_dir.mkdir(parents=True, exist_ok=True)

# determine filename from input json if available
if 'json_file' in locals() and json_file.exists():
    train_list_name = f"{json_file.stem}_train.txt"
else:
    train_list_name = "train.txt"
    print("json_file not found or defined, defaulting to 'train.txt'")

train_list_path = config_dir / train_list_name

# Set to True to force regeneration of the list (useful if paths changed)
force_regenerate = True 

train_list_path = generate_training_list(images_link, labels_link, train_list_path, force_regenerate=force_regenerate)

In [ ]:
# create dataset.yaml
from src.training.preparation import create_dataset_yaml

dataset_yaml_path = create_dataset_yaml(
    path=data_root,
    train_path=train_list_path,
    val_path=train_list_path,
    names={0: 'izutsumi', 1: 'izutsumi_face'}
)

!cat {dataset_yaml_path}

In [ ]:
from ultralytics import YOLO

# load model
model = YOLO("yolo11s.pt") 

# train model
# project points to where runs are saved
project_dir = catnip_data_root / "runs"

results = model.train(
    data=str(dataset_yaml_path),
    epochs=100,
    imgsz=640,
    project=str(project_dir),
    name="izutsumi_v1",
    device="cuda", # "mps", "cuda", "cpu"
    cache=True, 
    exist_ok=True
)

In [ ]:
# save Model to Bucket Models Directory
from src.training.preparation import save_best_model

project_dir = catnip_data_root / "runs"
target_model_dir = catnip_data_root / "models"

save_best_model(
    project_dir=project_dir,
    run_name="izutsumi_v1",
    target_dir=target_model_dir,
    target_name="yolo11_izutsumi_trained.pt"
)

In [ ]:
# evaluate
metrics = model.val()
print(metrics) 

In [5]:
from ultralytics import YOLO
project_dir = catnip_data_root / "runs"

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\rifusaki\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# run inference on the entire dataset
from ultralytics import YOLO
from src.output.output import save_inference_results

# load the trained model
model_path = project_dir / "izutsumi_v1" / "weights" / "best.pt"
model = YOLO(model_path)

# define output directory mapping to ./catnip-data/results/
results_dir = catnip_data_root / "results"
results_dir.mkdir(parents=True, exist_ok=True)

# Use the actual manga directory directly
inference_source = settings.paths.manga_dir

# Create a specific output directory for flattened results
output_dir = results_dir / "inference"
output_dir.mkdir(parents=True, exist_ok=True)

print(f"running inference on {inference_source}...")

# run prediction recursively
results = model.predict(
    source=str(inference_source) + "/**/*.*",  # recursive glob pattern
    project=str(results_dir),
    name="inference",
    save=False,      # Disable auto-save to handle manually
    save_txt=False,  # Disable auto-txt to handle manually
    conf=0.25,      # confidence threshold
    stream=True     # use generator to handle large datasets
)

save_inference_results(results, output_dir, inference_source)

running inference on catnip-data\data\manga...
saving flattened results to C:\Users\rifusaki\Desktop\catnip\catnip-data\results\inference

image 1/2752 C:\Users\rifusaki\Desktop\catnip\catnip-data\data\manga\v01\dan1001.jpg: 288x640 (no detections), 356.3ms
image 2/2752 C:\Users\rifusaki\Desktop\catnip\catnip-data\data\manga\v01\dan1002.jpg: 448x640 (no detections), 489.0ms
image 3/2752 C:\Users\rifusaki\Desktop\catnip\catnip-data\data\manga\v01\dan1003.jpg: 448x640 (no detections), 386.9ms
image 4/2752 C:\Users\rifusaki\Desktop\catnip\catnip-data\data\manga\v01\dan1004.jpg: 640x448 (no detections), 382.2ms
image 5/2752 C:\Users\rifusaki\Desktop\catnip\catnip-data\data\manga\v01\dan1005.jpg: 640x448 (no detections), 368.9ms
image 6/2752 C:\Users\rifusaki\Desktop\catnip\catnip-data\data\manga\v01\dan1006.jpg: 640x448 (no detections), 406.4ms
image 7/2752 C:\Users\rifusaki\Desktop\catnip\catnip-data\data\manga\v01\dan1007.jpg: 640x448 (no detections), 351.9ms
image 8/2752 C:\Users\rifusa